In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import src.data_etl as etl
import imp
imp.reload(etl);

In [3]:
funding_rounds = pd.read_csv('startup-investments/funding_rounds.csv',low_memory=False)
people = pd.read_csv('startup-investments/people.csv',low_memory=False)
objects = pd.read_csv('startup-investments/objects.csv',low_memory=False)
funds = pd.read_csv('startup-investments/funds.csv',low_memory=False)
degrees = pd.read_csv('startup-investments/degrees.csv',low_memory=False)
relationships = pd.read_csv('startup-investments/relationships.csv',low_memory=False)
offices = pd.read_csv('startup-investments/offices.csv',low_memory=False)
investments = pd.read_csv('startup-investments/investments.csv',low_memory=False)
milestones = pd.read_csv('startup-investments/milestones.csv',low_memory=False)
acquisitions = pd.read_csv('startup-investments/acquisitions.csv',low_memory=False)

In [63]:
acq_df, ipo_df , offices_df, relationship_df, fundraise_df = etl.load_data()

In [64]:
acq_df.head(1)

,name,object_id,price_currency_code,price_amount,acquiring_company,created_at
0,Flektor,c:10,USD,20000000.0,Fox Interactive Media,2007-05-31 22:19:54


In [65]:
ipo_df.head(1)

,name,object_id,stock_symbol,valuation_currency_code,valuation_amount,public_at
0,Apple,c:1654,NASDAQ:AAPL,USD,0.0,1980-12-19


In [66]:
acq_df['year'] = pd.to_datetime(acq_df['created_at']).apply(lambda x: x.year)
acq_df.drop('created_at',axis=1,inplace=True)

In [67]:
acq_df['exit_type'] = 'Acquired'

In [68]:
acq_df = acq_df[['object_id','exit_type','acquiring_company','year','price_currency_code','price_amount']]

In [69]:
acq_df.head(1)

,object_id,exit_type,acquiring_company,year,price_currency_code,price_amount
0,c:10,Acquired,Fox Interactive Media,2007,USD,20000000.0


In [70]:
ipo_df['year'] = pd.to_datetime(ipo_df['public_at']).apply(lambda x: x.year)

In [71]:
ipo_df['exit_type'] = 'IPO'

In [72]:
ipo_df = ipo_df[['object_id','exit_type','stock_symbol','year','valuation_currency_code','valuation_amount']]

In [73]:
acq_df.head(1)

,object_id,exit_type,acquiring_company,year,price_currency_code,price_amount
0,c:10,Acquired,Fox Interactive Media,2007,USD,20000000.0


In [74]:
ipo_df.head(1)

,object_id,exit_type,stock_symbol,year,valuation_currency_code,valuation_amount
0,c:1654,IPO,NASDAQ:AAPL,1980.0,USD,0.0


In [75]:
acq_df.rename(columns={'acquiring_company':'symbol/acquiring_company','price_currency_code':'currency','price_amount':'valuation'},inplace=True)

In [76]:
ipo_df.rename(columns={'stock_symbol':'symbol/acquiring_company','valuation_currency_code':'currency','valuation_amount':'valuation'},inplace=True)

In [77]:
acq_df.head(1)

,object_id,exit_type,symbol/acquiring_company,year,currency,valuation
0,c:10,Acquired,Fox Interactive Media,2007,USD,20000000.0


In [78]:
ipo_df.head(1)

,object_id,exit_type,symbol/acquiring_company,year,currency,valuation
0,c:1654,IPO,NASDAQ:AAPL,1980.0,USD,0.0


In [82]:
exits_df = pd.concat([acq_df,ipo_df])

In [106]:
exits_df

,object_id,exit_type,symbol/acquiring_company,year,currency,valuation
0,c:10,Acquired,Fox Interactive Media,2007.0,USD,20000000.0
1,c:72,Acquired,Google,2007.0,USD,60000000.0
2,c:132,Acquired,CBS,2007.0,USD,280000000.0
3,c:155,Acquired,Google,2007.0,USD,100000000.0
4,c:215,Acquired,Scripps Networks Interactive,2007.0,USD,25000000.0
...,...,...,...,...,...,...
1254,c:267859,IPO,NYSE:EBS,NaN,USD,0.0
1255,c:39330,IPO,NYSE:ARMK,2013.0,USD,0.0
1256,c:220208,IPO,NYSE:HLT,2008.0,USD,0.0
1257,c:243568,IPO,OTC:BCLI,NaN,USD,0.0


In [110]:
objects.columns

Index(['id', 'entity_type', 'entity_id', 'parent_id', 'name',
       'normalized_name', 'permalink', 'category_code', 'status', 'founded_at',
       'closed_at', 'domain', 'homepage_url', 'twitter_username', 'logo_url',
       'logo_width', 'logo_height', 'short_description', 'description',
       'overview', 'tag_list', 'country_code', 'state_code', 'city', 'region',
       'first_investment_at', 'last_investment_at', 'investment_rounds',
       'invested_companies', 'first_funding_at', 'last_funding_at',
       'funding_rounds', 'funding_total_usd', 'first_milestone_at',
       'last_milestone_at', 'milestones', 'relationships', 'created_by',
       'created_at', 'updated_at'],
      dtype='object')

In [116]:
outcomes = objects[['id','name','category_code','founded_at','funding_total_usd','status']].merge(exits_df,left_on='id',right_on='object_id',how='left')

In [122]:
outcomes['status'].value_counts()

operating      443663
acquired         9648
live             4349
closed           2773
ipo              1135
beta              780
development       226
private           219
alpha             113
Name: status, dtype: int64

In [142]:
relationship_df.drop(relationship_df[relationship_df['title'].isnull()].index,inplace=True)

In [146]:
relationship_df = relationship_df[relationship_df['title'].str.contains('Founder')][['person_id','id','first_name','last_name','title','name']]

In [148]:
relationship_df.drop_duplicates(inplace=True)

In [152]:
relationship_df['full_name'] = relationship_df.apply(lambda row: row['first_name'] + ' ' + row['last_name'],axis=1)

In [154]:
relationship_df.drop(['first_name','last_name'],inplace=True,axis=1)

In [158]:
outcomes = outcomes[['id','founded_at','funding_total_usd','status','exit_type','currency','valuation']]

In [159]:
outcomes.head(1)

,id,founded_at,funding_total_usd,status,exit_type,currency,valuation
0,c:1,2005-10-17,39750000.0,operating,NaN,NaN,NaN


In [163]:
relationship_df = relationship_df.merge(outcomes,left_on='id',right_on='id',how='left')

In [164]:
relationship_df.head(1)

,person_id,id,title,name,full_name,founded_at,funding_total_usd,status,exit_type,currency,valuation
0,p:2,c:1,Co-Founder/CEO/Board of Directors,Wetpaint,Ben Elowitz,2005-10-17,39750000.0,operating,NaN,NaN,NaN


In [189]:
founders = relationship_df.groupby('full_name').agg({'name':np.count_nonzero,'funding_total_usd':np.sum,'valuation':np.sum,'exit_type':np.count_nonzero}).reset_index()

In [191]:
founders.sort_values(by='funding_total_usd',ascending=False,inplace=True)

In [192]:
founders

,full_name,name,funding_total_usd,valuation,exit_type
9464,Craig O. McCaw,2,1.140000e+10,0.000000e+00,2
18686,Jack Dorsey,3,2.660833e+09,3.620000e+10,3
13116,Dustin Moskovitz,2,2.463900e+09,1.040000e+11,2
13340,Eduardo Saverin,1,2.425700e+09,1.040000e+11,1
28253,Mark Zuckerberg,2,2.425700e+09,1.040000e+11,2
...,...,...,...,...,...
18825,Jacques Blom,1,0.000000e+00,0.000000e+00,1
18826,Jacques Gravel,1,0.000000e+00,0.000000e+00,1
18827,Jacques Habra,1,0.000000e+00,0.000000e+00,1
18828,Jacques Kotze,1,0.000000e+00,0.000000e+00,1
